In [1]:
from Variables import *
from LoadFunctions import *
from ThreeDWindRadarFunctions import *
from MatchFunctions import *
from OtherUsefulFunctions import *
from PlotInteractiveFunctions import *
from InteractiveFunctions import *

from ClassificationFunctions import *
from Paths import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
from tkinter import *
import tkinter
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [3]:
chosenDate = '2020712'
mode = 'high'
beam_number = 5

optionPlot = 4

In [4]:
if '2019' in chosenDate:
    path_rain = pathRain[0]
elif '2020' in chosenDate:
    path_rain = pathRain[1]

# Load 
folder_radar, filepaths_radar, filenames_radar, date_radar, date_radardt,radarNoFile = loadRadar(chosenDate, path_dat)
folder_lidar, filepaths_lidar, filenames_lidar, date_lidar,date_lidardt,_,_,_,lidarNoFile = loadLidar(chosenDate, path_lidar,optionZip=False)

# Match
if (radarNoFile is False) and (lidarNoFile is False):
    lidarStart, lidarEnd, indiceMissing = lidarStartEnd(chosenDate, filepaths_lidar, reCreate=False)
    radarStart, radarEnd = radarStartEnd(chosenDate, filepaths_radar, reCreate=False) #m14 !!
    JoinedData = retrieveLidarRadialSpeed(filepaths_radar,filepaths_lidar, radarStart, radarEnd, lidarStart, lidarEnd, 
                                          radar2lidar, chosenDate, high_mode,low_mode, reCreate=False)

# Wind
path = path_highmode if mode == 'high' else path_lowmode

df_aswFlag,optionFlag = concatASWradar(chosenDate, path, optionFlag=True)
df_radialFlag = uvw2radial(df_aswFlag, beam_radar, elevationAngle=75, mode=mode)
df_aswFlag = ASW2Barb(df_aswFlag,optionFlag)

df_asw,optionFlag = concatASWradar(chosenDate, path, optionFlag=False)
df_radial = uvw2radial(df_asw, beam_radar, elevationAngle=75, mode=mode)
df_asw = ASW2Barb(df_asw,optionFlag)

if path_dat not in JoinedData['filename'][0]:
    print('Changing paths')
    JoinedData = replacePath(JoinedData, chosenDate, path_dat)

Loading pickle files : start and end times from the lidar for day 2020712
Loading pickle files : start and end times from the radar for day 2020712
Loading pickle files : matched radar and lidar acquisition times for day 2020712
Changing paths


In [5]:
def main():   
    app = InteractiveApp(None)
    app.protocol("WM_DELETE_WINDOW", disable_event)
    app.title('Trial')
    app.mainloop()
    return goOn

class InteractiveApp(Tk):
    def __init__(self, parent):
        Tk.__init__(self,parent)
        self.parent = parent
        self.widgets = []
        self.attributes('-zoomed', True)
        self.rowconfigure(1, weight=1)
        self.columnconfigure(1, weight=1)
        self.initialize(parent)
        
    def initialize(self, parent):                     
        
        im_frame = Frame(parent, width=500, height=400, background="#ffffff")
        im_frame.grid(row=0, column=1, rowspan=2, sticky="nsew") 
        
        self.dispim(fig,im_frame)
               
        self.left_frame = Frame(parent, width=150, bg="#ababab")
        self.left_frame.grid(row=0, column=0, rowspan=2, sticky="nsew")
        
        Label(self.left_frame, text="Choose gate number\nPress Ctrl + s to save your choice ", bg="#ababab").pack()
        vcmdgate = (self.register(self.onValidateGate),'%d', '%i', '%P', '%s', '%S', '%v', '%V', '%W')
        self.gateNb = Entry(self.left_frame, validate="key", validatecommand=vcmdgate) 
        self.gateNb.pack()
        self.button_gate = Button(self.left_frame, text = "", command = self.wgate)
        self.bind('<Control-KeyRelease-s>', lambda event=None: self.button_gate.invoke())
        Label(self.left_frame, text="\n", bg="#ababab").pack()        
        
        Label(self.left_frame, text="0 : no wind", bg="#ababab").pack()
        Label(self.left_frame, text="1 : only wind", bg="#ababab").pack()
        Label(self.left_frame, text="2 : no wind + contamination", bg="#ababab").pack()
        Label(self.left_frame, text="3 : wind + contamination", bg="#ababab").pack()
        Label(self.left_frame, text="4: rain", bg="#ababab").pack()
        Label(self.left_frame, text="5: no class", bg="#ababab").pack()
        
        Label(self.left_frame, text="Fill all entries with", bg="#ababab").pack()
              
        vcmd = (self.register(self.onValidate),'%d', '%i', '%P', '%s', '%S', '%v', '%V', '%W')
             
        self.common_number = StringVar()      
        self.common = Entry(self.left_frame, validate="key", validatecommand=vcmd, textvariable=self.common_number)        
        self.common.pack()
        self.after(100, lambda: self.common.focus())
        
        Label(self.left_frame, text="\n\n", bg="#ababab").pack()
                
        self.single_number = []
        for r, rdate in enumerate(date_toute1):
            self.single_number.append(StringVar())
            Label(self.left_frame, text="Time {0}".format(rdate), bg="#ababab").pack()
            self.common_number.trace("w", self.update_c) 
            #vcmd = (self.register(self.onValidate2),'%d', '%i', '%P', '%s', '%S', '%v', '%V', '%W')
            self.single = Entry(self.left_frame, validate="key", validatecommand=vcmd, textvariable=self.single_number[-1])
            
            if not np.isnan(df_save.iloc[gate][rdate]):
                self.single.insert(END, '{:.0f}'.format(df_save.iloc[gate][rdate]))     
            self.single.pack()  
            self.widgets.append(self.single)
       
        Label(self.left_frame, text="\n", bg="#ababab").pack()
        self.button_finished = Button(self.left_frame, text = "Finished \n(or press Enter)", command = self.wsave)
        self.button_finished.pack()
        self.bind('<Return>', lambda event=None: self.button_finished.invoke())
        
        Label(self.left_frame, text="\n\n", bg="#ababab").pack()
        self.button_stop = Button(self.left_frame, text = "STOP \n(or press Ctrl + c)", command = self.wstop)
        self.button_stop.pack()
        self.bind('<Control-KeyRelease-c>', lambda event=None: self.button_stop.invoke())
        
    def update_c(self,*args):
        for r, rdate in enumerate(date_toute2):
            self.single_number[r].set(self.common_number.get())
     
    def dispim(self, fig,im_frame):
        im = FigureCanvasTkAgg(fig, im_frame)      
        im.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)
             
    def wsave(self):
        global params
        params=[]
        for widget in self.widgets:
            params.append(widget.get())
        if '' in params:
            tkinter.messagebox.showinfo("Nice message","\n\nMissing at least one entry \n If you wanna stop, you need to fill those last ones ;)")
        else:
            self.destroy()

    def wstop(self):       
        self.wsave()
        #stop the loop
        global goOn
        goOn=False
        print('Bye Bye !')
   
    def onValidate(self, d, i, P, s, S, v, V, W):
        if V == 'forced':
            return True
        elif V == 'key':
            if S.isdigit() and int(i)==0 and int(S)<6:
                return True
            else:
                self.bell()
                return False
    
    def onValidateGate(self, d, i, P, s, S, v, V, W):
        #print(d, i, P, s, S, v, V, W)
        if S.isdigit() and int(i)<=1:
            return True
        else:
            self.bell()
            return False
  
    def wgate(self):
        if int(self.gateNb.get()) > 54:
            tkinter.messagebox.showinfo("Nice message","\n\nGate number not valid, need a number in between 0 and 54s\n\nDon't forget to press Ctrl + s again to save your new choice")
        else:
            global gate_Nb
            gate_Nb=int(self.gateNb.get())
            tkinter.messagebox.showinfo("New gate chosen saved","\n\nPlease finish to fill up what you have on the screen first")

In [6]:
# If you want to select different hours, you can chose them freely 
# The interval as well, does not need to be hourly. Longer becomes heavy though
start = ['0000','0100','0200', '0300', '0400']
end =   ['0100','0200','0300', '0400', '0500']
# You can also chose the gate and the incrementGate in the following lines

In [7]:
name, df_save, date_toute = interactiveManageFiles(date_radardt, chosenDate, beam_number)
df_save= interactivateFlagRainEvents(path_rain, chosenDate, df_save, name)

gate, column = interactiveFindLastClassified(df_save)
gate = 5
incrementGate=1
gate_Nb = gate
count=1
goOn = True

while goOn:
    if count ==1:
        ss = int(column[0:2]) 
        count += 1
    else :
        ss = 0
        
    for indice in range(0, len(start)):
        date_toute1, date_toute2,date_t, fig = plotAlltogether(chosenDate, start[indice], end[indice], int(gate), mode, int(beam_number),
                                radarStart, radarEnd, JoinedData, pathSave, df_radial,df_aswFlag, date_radardt,
                                optionSave=True, optionSpec=5, optionPlot=4,linewidth=1.0, fontsize1=18, fontsize2=12)
        for key in date_toute1:
            if key not in list(df_save.keys()):
                date_toute1.remove(key)
                date_toute2.remove(key[0:2]+':'+key[2:4])
        
        goOn = main()
        
        for index, date in enumerate(date_toute1): 
            df_save.at[gate,date]=params[index]
            
        df_save.to_csv(name, index = True)
        
        if goOn is False:
            break
        if gate_Nb!=gate:
            gate=gate_Nb-1
            break
       
    gate=gate+incrementGate
    gate_Nb = gate
    
    if gate == 55:
        print('Last gate finished')
        goOn = False
        break

Bye Bye !
